## Clustering the images using k-means

In [ ]:
filename = 'result-wse_flickr.jsonl'
filepath = '../tasks/03.image-crawl'

import os

filename = os.path.join(filepath, filename)
print filename

print os.path.exists(filename)

In [ ]:
# load the data
# only get the data for query='bench'

lines = open(filename).readlines()
print len(lines)

import json
items = [json.loads(line) for line in lines]
items = filter(lambda item: item.get('embeds') and item.get('query') == 'bird', items)
print len(items)

print items[0].keys()


In [ ]:
urls = [item['url'] for item in items]
features = [item['embeds'] for item in items]

print len(urls)
print len(features)

print urls[0]
print "dimension", len(features[0])
#print features[0]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from sklearn.cluster import KMeans
from sklearn import datasets

np.random.seed(5)

from time import time
import numpy as np
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

#digits = load_digits()
#data = scale(digits.data)


In [ ]:
# convert data to nparray
data = scale(np.array(features))


In [ ]:
data.shape

labels = np.array(urls)
print labels.shape


In [ ]:
print data.shape

n_samples, n_features = data.shape
k = 3

k_means = KMeans(init='k-means++', n_clusters=k, n_init=10)
k_means.fit(data)


In [ ]:
print(k_means.labels_[:100])


In [ ]:
# plot the top images in each cluster

%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

from PIL import Image
import requests
import StringIO

img_io = requests.get(urls[0][:10][0])
img = Image.open(StringIO.StringIO(img_io.content))
resized = img.thumbnail((80, 80), Image.ANTIALIAS)
print 'sss'
plt.imshow(img)
plt.show()

In [ ]:
clustered_urls = [[], [], []]

print clustered_urls

it = np.nditer(k_means.labels_, flags=['f_index'])

while not it.finished:
    #print it.index, it[0], urls[it.index]
    clustered_urls[it[0]].append(urls[it.index])
    #print clustered_urls
    it.iternext()

for idx in range(k):
    print 'count for %d: %d' % (idx, len(clustered_urls[idx]))

print clustered_urls[0][:10]
print
print clustered_urls[1][:10]


In [ ]:
# get the nearest neighbour pictures

import numpy
from scipy.spatial import distance

def sim_search_k_images(images_features, search_feature):
    """
    images_features is an array of the features
    search_feature is the feature to be searched for nearest neighbour
    
    use distance measure to get the k nearest neighbours
    and return the indices of the samples
    """

    idx = 0
    results = []
    for feature in images_features:
        
        if numpy.array_equal(feature, search_feature):
            idx += 1
            continue
        
        dst = distance.euclidean(search_feature, feature)
        results.append((idx, dst))
        idx += 1
        
    top_results = sorted(results, key=lambda x: x[1])[:9]
    top_ids = [image[0] for image in top_results]
    return top_ids


In [ ]:
# plot 3x3 picture matrix

import skimage.io as io
%matplotlib inline

from matplotlib.pyplot import figure, show, axes, sci
from matplotlib import cm, colors
from matplotlib.font_manager import FontProperties
from numpy import amin, amax, ravel
from numpy.random import rand

def plot_3x3(images, title = ''):
    """
    given 9 images, plot them in 3x3 matrix
    """

    img_bytes = []
    for image in images:
        img_io = requests.get(image)
        
        img = io.imread(StringIO.StringIO(img_io.content))
        img_bytes.append(img)
        """
        io.imshow(img)
        io.show()
        """

    Nr = 3
    Nc = 3

    fig = figure()
    cmap = cm.cool

    figtitle = title
    t = fig.text(0.70, 1.50, figtitle,
                 horizontalalignment='center',
                 fontproperties=FontProperties(size=16))

    cax = fig.add_axes([0.2, 0.08, 0.6, 0.04])

    w = 0.4
    h = 0.4
    ax = []
    images = []
    for i in range(Nr):
        for j in range(Nc):
            pos = [0.075 + j*1.1*w, 0.18 + i*1.1*h, w, h]
            a = fig.add_axes(pos)
            if i > 0:
                a.set_xticklabels([])

            # Make some fake data with a range that varies
            # somewhat from one plot to the next.
            data = img_bytes.pop(0)

            images.append(a.imshow(data, cmap=cmap))

            ax.append(a)

    show()

In [ ]:
# show the top 9 images for each cluster

import skimage.io as io
%matplotlib inline

for cat in range(3):
    img_urls = clustered_urls[cat][:9]
    plot_3x3(img_urls, title = '9 samples for cluster %d' % cat)
    print '---------------------'


In [ ]:
ids = sim_search_k_images(data, data[222])
print ids

img_urls = [urls[id] for id in ids]
print img_urls

plot_3x3(img_urls)